In [4]:
!pip install -q nltk
!pip install -q spacy 
!pip install -q tabulate

In [15]:
!python -m spacy download en

     -------------------------------------- 12.8/12.8 MB 603.0 kB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-11-05 18:10:21.422291: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-11-05 18:10:21.422334: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-05 18:10:24.357041: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-11-05 18:10:24.358765: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-11-05 18:10:24.360417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-11-05 18:10:24.362064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jaskirat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jaskirat\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [19]:
from collections import Counter
import spacy
from tabulate import tabulate
nlp = spacy.load('en_core_web_sm')

## Pipeline Flow: 
### Initial Steps:
- Sentence Tokenization
- Word Tokenization
- Part-of-Speech Tagging
- Extracting nouns and verbs

In [9]:
text = u"""
Dealing with textual data is very crucial so to handle these text data we need some 
basic text processing steps. Most of the processing steps covered in this section are 
commonly used in NLP and involve the combination of several steps into a single 
executable flow. This is usually referred to as the NLP pipeline. These flow 
can be a combination of tokenization, stemming, word frequency, parts of 
speech tagging, etc.
"""

sentences = nltk.sent_tokenize(text)

words = [nltk.word_tokenize(s) for s in sentences]

tagged_wt = [nltk.pos_tag(w)for w in words]

patternPOS= []
for tag in tagged_wt:
    patternPOS.append([v for k,v in tag])

nouns = []  
for tag in tagged_wt:
    nouns.append([k for k,v in tag if v in ['NN','NNS','NNP','NNPS']])

verbs = []  
for tag in tagged_wt:
    verbs.append([k for k,v in tag if v in ['VB','VBD','VBG','VBN','VBP','VBZ']])

In [12]:
print("Sentences: ", sentences)
print("\n\nWords: ", words)
print("\n\nPOS: ", tagged_wt)
print("\n\nPOS pattern: ",patternPOS)
print("\n\nExtracted nouns: ",nouns)
print("\n\nExtracted verbs: ",verbs)

Sentences:  ['\nDealing with textual data is very crucial so to handle these text data we need some \nbasic text processing steps.', 'Most of the processing steps covered in this section are \ncommonly used in NLP and involve the combination of several steps into a single \nexecutable flow.', 'This is usually referred to as the NLP pipeline.', 'These flow \ncan be a combination of tokenization, stemming, word frequency, parts of \nspeech tagging, etc.']


Words:  [['Dealing', 'with', 'textual', 'data', 'is', 'very', 'crucial', 'so', 'to', 'handle', 'these', 'text', 'data', 'we', 'need', 'some', 'basic', 'text', 'processing', 'steps', '.'], ['Most', 'of', 'the', 'processing', 'steps', 'covered', 'in', 'this', 'section', 'are', 'commonly', 'used', 'in', 'NLP', 'and', 'involve', 'the', 'combination', 'of', 'several', 'steps', 'into', 'a', 'single', 'executable', 'flow', '.'], ['This', 'is', 'usually', 'referred', 'to', 'as', 'the', 'NLP', 'pipeline', '.'], ['These', 'flow', 'can', 'be', '

In [20]:
doc = nlp(text)
noun_counter = Counter(token.lemma_ for token in doc if token.pos_ == 'NOUN')

print(tabulate(noun_counter.most_common(5), headers=['Noun', 'Count']))

Noun           Count
-----------  -------
step               3
datum              2
text               2
processing         2
combination        2


### Dependency Parsing

In [22]:
doc = nlp(sentences[2])
spacy.displacy.render(doc,style='dep', options={'distance' : 140}, jupyter=True)

### Named Entity Recognition (NER)

In [32]:
# doc = nlp("Hey there! Jaskirat this side. I like NLP.")
doc = nlp(sentences[1])
entity_types = ((ent.text, ent.label_) for ent in doc.ents)
print(tabulate(entity_types, headers=['Entity', 'Entity Type']))
print()
token_entity_info = ((token.text, token.ent_iob_, token.ent_type_,) for token in doc)
print(tabulate(token_entity_info, headers=['Token', 'IOB Annotation', 'Entity Type']))

Entity    Entity Type
--------  -------------
NLP       ORG

Token        IOB Annotation    Entity Type
-----------  ----------------  -------------
Most         O
of           O
the          O
processing   O
steps        O
covered      O
in           O
this         O
section      O
are          O
             O
commonly     O
used         O
in           O
NLP          B                 ORG
and          O
involve      O
the          O
combination  O
of           O
several      O
steps        O
into         O
a            O
single       O
             O
executable   O
flow         O
.            O


In [33]:
doc = nlp("My name is Jaskirat and I am a CS undergrad.")

entity_types = ((ent.text, ent.label_) for ent in doc.ents)
print(tabulate(entity_types, headers=['Entity', 'Entity Type']))

Entity    Entity Type
--------  -------------
Jaskirat  PERSON
CS        ORG
